In [7]:
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
DOMAINS = [
    "blocksworld",
    "ferry",
    "miconic",
    # "rovers",
    "satellite",
    # "transport",
]

In [5]:
baseline_data = pd.read_csv('baseline_results.csv')
baseline_data

,domain,problem,repeat,time,plan_length,plan_found
0,blocksworld,0_01,0,1.410242,10.0,True
1,blocksworld,0_01,1,1.413986,10.0,True
2,blocksworld,0_01,2,1.516747,10.0,True
3,blocksworld,0_01,3,1.534240,10.0,True
4,blocksworld,0_01,4,1.516963,10.0,True
...,...,...,...,...,...,...
3595,satellite,2_30,5,NaN,NaN,False
3596,satellite,2_30,6,NaN,NaN,False
3597,satellite,2_30,7,NaN,NaN,False
3598,satellite,2_30,8,NaN,NaN,False


In [9]:
for domain in DOMAINS:
    print(domain)
    data = baseline_data[baseline_data['domain'] == domain]
    fig = px.scatter(baseline_data, x="problem", y="plan_length")
    fig.show()

blocksworld


ferry


miconic


satellite
